In [ ]:
!pip install gensim==3.8.3
!pip install keras --upgrade
!pip install pandas --upgrade
!pip install tensorflow --upgrade

In [2]:
import pandas as pd

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

In [3]:
vocab_size = 32965
# WORD2VEC 
W2V_SIZE = 300
SEQUENCE_LENGTH = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

In [4]:
# DATASET
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
TRAIN_SIZE = 0.8

# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 10
BATCH_SIZE = 1024

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)

# EXPORT
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.w2v"
TOKENIZER_MODEL = "tokenizer.pkl"
ENCODER_MODEL = "encoder.pkl"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#LOADING
load_dir = '/content/drive/MyDrive/nns/'
train_test_dir = load_dir+'saved_train_test/'

embedding_matrix = np.load(load_dir+'embedding_matrix.npy')

x_train = np.load(train_test_dir+'x_train.npy')
y_train = np.load(train_test_dir+'y_train.npy')

x_test = np.load(train_test_dir+'x_test.npy')
y_test = np.load(train_test_dir+'y_test.npy')

In [7]:
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print()
print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train (40000, 300)
y_train (40000, 1)

x_test (10000, 300)
y_test (10000, 1)


###NN Model

In [8]:
#HYPER PARAMETERS
model_name = "RNN"
num_epochs = 10
batch_size = 1024
rnn_units = 784
sequence_length=300
activation="sigmoid"
optimizer='adam'

In [9]:
#MODEL
embedding_layer = Embedding(vocab_size, W2V_SIZE, weights=[embedding_matrix], 
                            input_length=SEQUENCE_LENGTH, 
                            trainable=False)

model = Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.GRU(rnn_units,
                        recurrent_initializer='glorot_uniform'))
model.add(Dense(1, activation='sigmoid'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          9889500   
                                                                 
 dropout (Dropout)           (None, 300, 300)          0         
                                                                 
 gru (GRU)                   (None, 784)               2554272   
                                                                 
 dense (Dense)               (None, 1)                 785       
                                                                 
Total params: 12,444,557
Trainable params: 2,555,057
Non-trainable params: 9,889,500
_________________________________________________________________


In [10]:
#OPTIMIZATION
callbacks = [ tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
#TRAINING
%%time
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/10
36/36 [==============================] - 62s 2s/step - loss: 0.5835 - accuracy: 0.6903 - val_loss: 0.5461 - val_accuracy: 0.7207 - lr: 0.0010
Epoch 2/10
36/36 [==============================] - 57s 2s/step - loss: 0.5493 - accuracy: 0.7179 - val_loss: 0.5338 - val_accuracy: 0.7287 - lr: 0.0010
Epoch 3/10
36/36 [==============================] - 57s 2s/step - loss: 0.5351 - accuracy: 0.7274 - val_loss: 0.5295 - val_accuracy: 0.7278 - lr: 0.0010
Epoch 4/10
36/36 [==============================] - 56s 2s/step - loss: 0.5260 - accuracy: 0.7319 - val_loss: 0.5285 - val_accuracy: 0.7265 - lr: 0.0010
Epoch 5/10
36/36 [==============================] - 58s 2s/step - loss: 0.5193 - accuracy: 0.7377 - val_loss: 0.5283 - val_accuracy: 0.7327 - lr: 0.0010
Epoch 6/10
36/36 [==============================] - 56s 2s/step - loss: 0.5162 - accuracy: 0.7393 - val_loss: 0.5426 - val_accuracy: 0.7203 - lr: 0.0010
Epoch 7/10
36/36 [==============================] - 57s 2s/step - loss: 0.5143 - a

In [12]:
#TESTING
score = model.evaluate(x_test, y_test, batch_size=batch_size)

print("ACCURACY:",score[1])
print("LOSS:",score[0])

acc = history.history['accuracy']
print("acc:", acc)
val_acc = history.history['val_accuracy']
print("val_acc:", val_acc)
loss = history.history['loss']
print("loss:", loss)
val_loss = history.history['val_loss']
print("val_loss:", val_loss)


10/10 [==============================] - 4s 395ms/step - loss: 0.5218 - accuracy: 0.7407
ACCURACY: 0.7407000064849854
LOSS: 0.5218129754066467
acc: [0.6903055310249329, 0.7178611159324646, 0.7273610830307007, 0.7318888902664185, 0.7376944422721863, 0.7392500042915344, 0.7429166436195374, 0.7503888607025146, 0.7567777633666992, 0.7680833339691162]
val_acc: [0.7207499742507935, 0.7287499904632568, 0.7277500033378601, 0.7264999747276306, 0.7327499985694885, 0.7202500104904175, 0.7329999804496765, 0.7327499985694885, 0.7352499961853027, 0.7325000166893005]
loss: [0.5834757685661316, 0.5493130683898926, 0.5350783467292786, 0.5260161757469177, 0.5192914009094238, 0.5161949396133423, 0.5143142938613892, 0.49901577830314636, 0.48869064450263977, 0.4769092798233032]
val_loss: [0.5460783243179321, 0.5338124632835388, 0.5294843912124634, 0.5284732580184937, 0.5282540321350098, 0.5425861477851868, 0.5271004438400269, 0.5302062034606934, 0.5315521955490112, 0.5403430461883545]


In [13]:
#SAVING 
save_dir = '/content/drive/MyDrive/nns/saved_nn_models/'
model_name = "RNN"
model.save(save_dir+model_name+".h5")


In [14]:
!ls /content/drive/MyDrive/nns/saved_nn_models/

FFNN1.h5  FFNN3.h5  FFNN5.h5  LSTM.h5  RNN.h5
